In [ ]:
from dotenv import load_dotenv

from langchain_teddynote import logging
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel

In [ ]:
load_dotenv()

In [ ]:
logging.langsmith("langchain-01")  # 프로젝트 이름

In [ ]:
llm = ChatOpenAI()

prompt = PromptTemplate.from_template("{topic} 에 대하여 3문장으로 설명해줘.")

chain = prompt | llm | StrOutputParser()

In [ ]:
# chain.stream 메서드를 사용하여 '멀티모달' 토픽에 대한 스트림을 생성하고 반복합니다.
for token in chain.stream({"topic": "멀티모달"}):
    # 스트림에서 받은 데이터의 내용을(content) 출력합니다. 
    # end: 줄바꿈 없이 이어서 출력하고, 
    # flush=True: 버퍼를 즉시 비웁니다.
    print(token, end="", flush=True)

In [ ]:
chain.invoke({"topic": "ChatGPT"})

Batch: 단위 실행

In [ ]:
chain.batch(  # 딕셔너리 여러 개를 포함하는 리스트로 인자로 받아, 각 딕셔너리의 topic 키 값을 사용하여 일괄 처리
    [
        {"topic": "ChatGPT"},
        {"topic": "Instagram"},
        {"topic": "멀티모달"},
        {"topic": "프로그래밍"},
        {"topic": "머신러닝"},
    ],
    config={"max_concurrency": 3},  # 동시에 처리할 수 있는 최대 작업 수
)

async

In [ ]:
# async stream: 비동기 스트림

async for token in chain.astream({"topic": "YouTube"}):  # 스트림에서 메시지를 순차적으로 받아옴
    # 메시지 내용을(s.content) 출력합니다.
    # end="": 각 메시지를 줄바꿈 없이 바로 출력하고 
    # flush=True: 출력 버퍼를 비워 지체 없이 화면에 나타나도록 합니다. (버퍼: 데이터를 임시로 보관하는 저장 공간)
    print(token, end="", flush=True)

In [ ]:
# async invoke: 비동기 호출

a_invoke = chain.ainvoke({"topic": "NVDA"})  # 특정 주제에 대한 처리를 비동기적으로 요청

await a_invoke

In [ ]:
# async batch: 비동기 배치

a_batch = chain.abatch(
    [
        {"topic": "YouTube"}, 
        {"topic": "Instagram"}, 
        {"topic": "Facebook"}
    ]
)

await a_batch

병렬 체인 구성하기

In [ ]:
# {country} 의 수도를 물어보는 체인
chain1 = (
    PromptTemplate.from_template("{country} 의 수도는 어디야?")
    | llm
    | StrOutputParser()
)

# {country} 의 면적을 물어보는 체인
chain2 = (
    PromptTemplate.from_template("{country} 의 면적은 얼마야?")
    | llm
    | StrOutputParser()
)

In [ ]:
combined = RunnableParallel(capital=chain1, area=chain2)

In [ ]:
chain1.invoke({"country": "대한민국"})

In [ ]:
chain2.invoke({"country": "미국"})

In [ ]:
combined.invoke({"country": "대한민국"})

배치 병렬 처리

In [ ]:
chain1.batch([{"country": "대한민국"}, {"country": "미국"}])

In [ ]:
chain2.batch([{"country": "대한민국"}, {"country": "미국"}])

In [ ]:
combined.batch([{"country": "대한민국"}, {"country": "미국"}])